# Here a section for pre-processing data scraped

In [26]:
import json
import pandas as pd
from datetime import datetime
import spacy
import re


In [9]:
# Charger le fichier JSON
with open("data_scraped.json", "r", encoding="utf-8") as f:
    data = json.load(f)

rows = []

for publication in data:
    # Nettoyage du texte de la publication
    post_text = "\n".join([p.strip() for p in publication.get("post", []) if p.strip()])
    
    # Format date (facultatif)
    raw_date = publication.get("date_publication", "").replace("Publié le ", "")
    
    # Ajouter la publication principale
    rows.append({
        "type": "publication",
        "date": raw_date,
        "texte": post_text,
        "source_url": publication.get("url", "")
    })

    # Ajouter les commentaires et replies
    for comment in publication.get("comments", []):
        comment_text = comment.get("text", "").strip()
        comment_date = comment.get("date", "").strip()
        
        rows.append({
            "type": "comment",
            "date": comment_date,
            "texte": comment_text,
            "source_url": publication.get("url", "")
        })

        for reply in comment.get("replies", []):
            reply_text = reply.get("text", "").strip()
            reply_date = reply.get("date", "").strip()
            
            rows.append({
                "type": "reply",
                "date": reply_date,
                "texte": reply_text,
                "source_url": publication.get("url", "")
            })

# Convertir en DataFrame
df = pd.DataFrame(rows)

In [10]:
df

,type,date,texte,source_url
0,publication,dimanche 6 avril 2025 à 16h51min,"(Hèrèdougou,le 06 Avril 2025), Sous un soleil ...",https://lefaso.net/spip.php?article137232
1,comment,6 avril 19:07,Félicitations Mr le ministre pour votre présen...,https://lefaso.net/spip.php?article137232
2,reply,7 avril 16:53,Bravo à l’engagement du ministre Adama Luc SOR...,https://lefaso.net/spip.php?article137232
3,publication,jeudi 3 avril 2025 à 21h50min,"Comme prévu, le procès du groupe Breba Finance...",https://lefaso.net/spip.php?article137192
4,comment,4 avril 08:34,Je compatis au désarrois des victimes de ces v...,https://lefaso.net/spip.php?article137192
...,...,...,...,...
6941,comment,16 juillet 2024 20:14,"Mr @ Tranntoll, rassurez-vous car je n’ai aucu...",https://lefaso.net/spip.php?article131598
6942,comment,17 juillet 2024 02:21,@Sabrina\n\nUltra Faux ce que vous écrivez. Ar...,https://lefaso.net/spip.php?article131598
6943,comment,17 juillet 2024 12:52,"@AMKOULEL\n\nMessire AMKOULEL,\n\nd’abord, bon...",https://lefaso.net/spip.php?article131598
6944,comment,18 juillet 2024 00:35,@Renault HÉLIE\n\nIncroyable de constater votr...,https://lefaso.net/spip.php?article131598


## I-Pre-Processing

In [3]:
!pip install spacy

In [4]:
!python -m spacy download fr_core_news_sm

     ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
      --------------------------------------- 0.3/16.3 MB 6.7 MB/s eta 0:00:03
     - -------------------------------------- 0.7/16.3 MB 8.9 MB/s eta 0:00:02
     --- ------------------------------------ 1.6/16.3 MB 12.7 MB/s eta 0:00:02
     ----- ---------------------------------- 2.4/16.3 MB 12.9 MB/s eta 0:00:02
     ----- ---------------------------------- 2.4/16.3 MB 12.9 MB/s eta 0:00:02
     --------- ------------------------------ 3.8/16.3 MB 13.4 MB/s eta 0:00:01
     ------------ --------------------------- 5.0/16.3 MB 15.2 MB/s eta 0:00:01
     ------------ --------------------------- 5.1/16.3 MB 14.9 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/16.3 MB 14.2 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/16.3 MB 14.2 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/16.3 MB 14.2 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/

In [11]:
# Model loading
nlp = spacy.load("fr_core_news_sm")

In [27]:
def preprocess_text(text):
    doc = nlp(text)

    tokens = [
        token.text.lower() 
        for token in doc 
        if not token.is_punct and not token.is_space and not re.search(r'[@#]', token.text)
    ]
    
    return " ".join(tokens)

In [28]:
df["text_processed"] = df["texte"].apply(preprocess_text) #preprocess of text

In [29]:
df

,type,date,texte,source_url,text_processed,date_normalized
0,publication,dimanche 6 avril 2025 à 16h51min,"(Hèrèdougou,le 06 Avril 2025), Sous un soleil ...",https://lefaso.net/spip.php?article137232,hèrèdougou le 06 avril 2025 sous un soleil ard...,2025-04-06 16:51:00
1,comment,6 avril 19:07,Félicitations Mr le ministre pour votre présen...,https://lefaso.net/spip.php?article137232,félicitations mr le ministre pour votre présen...,2024-04-06 19:07:00
2,reply,7 avril 16:53,Bravo à l’engagement du ministre Adama Luc SOR...,https://lefaso.net/spip.php?article137232,bravo à l’ engagement du ministre adama luc so...,2024-04-07 16:53:00
3,publication,jeudi 3 avril 2025 à 21h50min,"Comme prévu, le procès du groupe Breba Finance...",https://lefaso.net/spip.php?article137192,comme prévu le procès du groupe breba finance ...,2025-04-03 21:50:00
4,comment,4 avril 08:34,Je compatis au désarrois des victimes de ces v...,https://lefaso.net/spip.php?article137192,je compatis au désarrois des victimes de ces v...,2024-04-04 08:34:00
...,...,...,...,...,...,...
6941,comment,16 juillet 2024 20:14,"Mr @ Tranntoll, rassurez-vous car je n’ai aucu...",https://lefaso.net/spip.php?article131598,mr tranntoll rassurez -vous car je n’ ai aucun...,2024-07-16 20:14:00
6942,comment,17 juillet 2024 02:21,@Sabrina\n\nUltra Faux ce que vous écrivez. Ar...,https://lefaso.net/spip.php?article131598,ultra faux ce que vous écrivez arrêtez vos thé...,2024-07-17 02:21:00
6943,comment,17 juillet 2024 12:52,"@AMKOULEL\n\nMessire AMKOULEL,\n\nd’abord, bon...",https://lefaso.net/spip.php?article131598,messire amkoulel d’ abord bonsoir chez vous ai...,2024-07-17 12:52:00
6944,comment,18 juillet 2024 00:35,@Renault HÉLIE\n\nIncroyable de constater votr...,https://lefaso.net/spip.php?article131598,hélie incroyable de constater votre changement...,2024-07-18 00:35:00


#### Normalization of Date to ISO format

In [11]:
!pip install dateparser

In [30]:
import datetime
import dateparser


def normaliser_date(date_str, reference_year=None):
    # Nettoyage manuel
    clean_date = date_str.lower()
    clean_date = re.sub(r'publié le ', '', clean_date)
    clean_date = re.sub(r'à ', '', clean_date)
    clean_date = re.sub(r'min', '', clean_date)
    clean_date = clean_date.strip()

    # Settings pour dateparser
    settings = {'PREFER_DATES_FROM': 'past', 'DATE_ORDER': 'DMY'}

    if reference_year:
        settings['RELATIVE_BASE'] = datetime.datetime(reference_year, 1, 1)

    parsed = dateparser.parse(clean_date, settings=settings)

    if parsed:
        return parsed.strftime("%Y-%m-%d %H:%M:%S")
    else:
        return None

In [31]:
# Application à une série de dates
df["date_normalized"] = df["date"].apply(lambda x: normaliser_date(x, reference_year=2025))

In [32]:
df

,type,date,texte,source_url,text_processed,date_normalized
0,publication,dimanche 6 avril 2025 à 16h51min,"(Hèrèdougou,le 06 Avril 2025), Sous un soleil ...",https://lefaso.net/spip.php?article137232,hèrèdougou le 06 avril 2025 sous un soleil ard...,2025-04-06 16:51:00
1,comment,6 avril 19:07,Félicitations Mr le ministre pour votre présen...,https://lefaso.net/spip.php?article137232,félicitations mr le ministre pour votre présen...,2024-04-06 19:07:00
2,reply,7 avril 16:53,Bravo à l’engagement du ministre Adama Luc SOR...,https://lefaso.net/spip.php?article137232,bravo à l’ engagement du ministre adama luc so...,2024-04-07 16:53:00
3,publication,jeudi 3 avril 2025 à 21h50min,"Comme prévu, le procès du groupe Breba Finance...",https://lefaso.net/spip.php?article137192,comme prévu le procès du groupe breba finance ...,2025-04-03 21:50:00
4,comment,4 avril 08:34,Je compatis au désarrois des victimes de ces v...,https://lefaso.net/spip.php?article137192,je compatis au désarrois des victimes de ces v...,2024-04-04 08:34:00
...,...,...,...,...,...,...
6941,comment,16 juillet 2024 20:14,"Mr @ Tranntoll, rassurez-vous car je n’ai aucu...",https://lefaso.net/spip.php?article131598,mr tranntoll rassurez -vous car je n’ ai aucun...,2024-07-16 20:14:00
6942,comment,17 juillet 2024 02:21,@Sabrina\n\nUltra Faux ce que vous écrivez. Ar...,https://lefaso.net/spip.php?article131598,ultra faux ce que vous écrivez arrêtez vos thé...,2024-07-17 02:21:00
6943,comment,17 juillet 2024 12:52,"@AMKOULEL\n\nMessire AMKOULEL,\n\nd’abord, bon...",https://lefaso.net/spip.php?article131598,messire amkoulel d’ abord bonsoir chez vous ai...,2024-07-17 12:52:00
6944,comment,18 juillet 2024 00:35,@Renault HÉLIE\n\nIncroyable de constater votr...,https://lefaso.net/spip.php?article131598,hélie incroyable de constater votre changement...,2024-07-18 00:35:00


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6946 entries, 0 to 6945
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   type             6946 non-null   object
 1   date             6946 non-null   object
 2   texte            6946 non-null   object
 3   source_url       6946 non-null   object
 4   text_processed   6946 non-null   object
 5   date_normalized  6946 non-null   object
dtypes: object(6)
memory usage: 325.7+ KB


In [34]:
df.to_csv("data_processed/dataset_nlp.csv", index=False, encoding="utf-8")